![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_MPNetForTokenClassification.ipynb)

## Import ONNX MPNetForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `MPNetForTokenClassification` is only available since in `Spark NLP 5.3.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import MPNet models trained/fine-tuned for token classification via `MPNetForTokenClassification` or `TFMPNetForTokenClassification`. These models are usually under `Token Classification` category and have `mpnet` in their labels
- Reference: [TFMPNetForTokenClassification](https://huggingface.co/docs/transformers/en/model_doc/mpnet#transformers.TFMPNetForTokenClassification)
- Some [example models](https://huggingface.co/models?filter=mpnet&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [hf-tiny-model-private/tiny-random-MPNetForTokenClassification](https://huggingface.co/hf-tiny-model-private/tiny-random-MPNetForTokenClassification) model from HuggingFace as an example
- In addition to `TFMPNetForTokenClassification` we also need to save the `MPNetTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from optimum.onnxruntime import ORTModelForTokenClassification
import tensorflow as tf

MODEL_NAME = 'hf-tiny-model-private/tiny-random-MPNetForTokenClassification'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

Framework not specified. Using pt to export the model.


pytorch_model.bin:   0%|          | 0.00/957k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/4.68k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/22.3k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu121


Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {ONNX_MODEL}

total 1056
-rw-r--r-- 1 root root     646 Feb 27 18:22 config.json
-rw-r--r-- 1 root root 1033329 Feb 27 18:22 model.onnx
-rw-r--r-- 1 root root     280 Feb 27 18:22 special_tokens_map.json
-rw-r--r-- 1 root root    1396 Feb 27 18:22 tokenizer_config.json
-rw-r--r-- 1 root root   22260 Feb 27 18:22 tokenizer.json
-rw-r--r-- 1 root root    4682 Feb 27 18:22 vocab.txt


In [4]:
!mkdir {ONNX_MODEL}/assets

- As you can see, we need to move `vocab.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [5]:
# get label2id dictionary
labels = ort_model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(ONNX_MODEL +'/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [8]:
!mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [9]:
!ls -lR {ONNX_MODEL}

onnx_models/hf-tiny-model-private/tiny-random-MPNetForTokenClassification:
total 1052
drwxr-xr-x 2 root root    4096 Feb 27 18:24 assets
-rw-r--r-- 1 root root     646 Feb 27 18:22 config.json
-rw-r--r-- 1 root root 1033329 Feb 27 18:22 model.onnx
-rw-r--r-- 1 root root     280 Feb 27 18:22 special_tokens_map.json
-rw-r--r-- 1 root root    1396 Feb 27 18:22 tokenizer_config.json
-rw-r--r-- 1 root root   22260 Feb 27 18:22 tokenizer.json

onnx_models/hf-tiny-model-private/tiny-random-MPNetForTokenClassification/assets:
total 12
-rw-r--r-- 1 root root   15 Feb 27 18:22 labels.txt
-rw-r--r-- 1 root root 4682 Feb 27 18:22 vocab.txt


## Import and Save DeBertaForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.0
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.8/564.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 22.9 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `MPNetForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `MPNetForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

tokenClassifier = MPNetForTokenClassification.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your MPNetForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 721576
-rw-r--r-- 1 root root 736413822 Jan  4 17:57 deberta_classification_onnx
-rw-r--r-- 1 root root   2464616 Jan  4 17:57 deberta_spp
drwxr-xr-x 3 root root      4096 Jan  4 17:57 fields
drwxr-xr-x 2 root root      4096 Jan  4 17:57 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DeBertaForTokenClassification model 😊

In [ ]:
tokenClassifier_loaded = MPNetForTokenClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
tokenClassifier_loaded.getClasses()

['I-UNIT',
 'U-PHYSICAL_QUALITY',
 'L-COLOR',
 'U-UNIT',
 'U-COLOR',
 'I-QUANTITY',
 'B-FOOD',
 'U-PART',
 'U-TASTE',
 'U-PROCESS',
 'L-PURPOSE',
 'I-PART',
 'I-PURPOSE',
 'B-PURPOSE',
 'L-PROCESS',
 'L-QUANTITY',
 'B-UNIT',
 'B-QUANTITY',
 'U-FOOD',
 'B-PART',
 'I-TASTE',
 'B-TASTE',
 'L-TASTE',
 'L-FOOD',
 'I-PROCESS',
 'L-PHYSICAL_QUALITY',
 'B-PHYSICAL_QUALITY',
 'I-PHYSICAL_QUALITY',
 'I-FOOD',
 'B-PROCESS',
 'B-COLOR',
 'O',
 'L-PART',
 'L-UNIT',
 'U-QUANTITY']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["I love spicy food"]]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show()

+-----------------+--------------------+
|             text|              result|
+-----------------+--------------------+
|I love spicy food|[O, O, U-TASTE, L...|
+-----------------+--------------------+



That's it! You can now go wild and use hundreds of `MPNetForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
